In [73]:
!pip install pandas
!pip install fairlearn
!pip install shap




[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/aa/7c/eead607a358b2466b95b302e4a7eae717eeed53615653fbbd4c1527041b9/shap-0.48.0-cp311-cp311-win_amd64.whl.metadata
  Using cached shap-0.48.0-cp311-cp311-win_amd64.whl.metadata (25 kB)
  Obtaining dependency information for numba>=0.54 from https://files.pythonhosted.org/packages/0f/a4/2b309a6a9f6d4d8cfba583401c7c2f9ff887adb5d54d8e2e130274c0973f/numba-0.61.2-cp311-cp311-win_amd64.whl.metadata
  Using cached numba-0.61.2-cp311-cp311-win_amd64.whl.metadata (2.9 kB)
  Obtaining dependency information for llvmlite<0.45,>=0.44.0dev0 from https://files.pythonhosted.org/packages/5f/c6/258801143975a6d09a373f2641237992496e15567b907a4d401839d671b8/llvmlite-0.44.0-cp311-cp311-win_amd64.whl.metadata
  Using cached llvmlite-0.44.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
Using cached shap-0.48.0-cp311-cp311-win_amd64.whl (544 kB)
Using cached numba-0.61.2-cp311-cp311-win_amd64.whl (2.8 MB)
Using cached llvmli


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, false_positive_rate

In [ ]:


# Load your dataset (example)
df = pd.read_csv('data\HRAttrition-Revised.csv')

# Features and target
target = 'Attrition'  # binary target, e.g., 1 if attrition, 0 otherwise
sensitive_cols = ['gender', 'religion', 'race', 'marital_status']



In [12]:
print("Column labels in the dataset in column order:")
for column in df.columns:
    print(column)

Column labels in the dataset in column order:
Age
Attrition
BusinessTravel
DailyRate
Department
DistanceFromHome
Education
EducationField
EnvironmentSatisfaction
Gender
HourlyRate
JobInvolvement
JobLevel
JobRole
JobSatisfaction
MaritalStatus
MonthlyIncome
MonthlyRate
NumCompaniesWorked
OverTime
PercentSalaryHike
PerformanceRating
RelationshipSatisfaction
StockOptionLevel
TotalWorkingYears
TrainingTimesLastYear
WorkLifeBalance
YearsAtCompany
YearsInCurrentRole
YearsSinceLastPromotion
YearsWithCurrManager
gender
religion
race
marital_status


In [22]:
from sklearn.preprocessing import LabelEncoder


# Encode categorical columns in df
categorical_cols_df = df.select_dtypes(include=['object', 'category']).columns.tolist()
if categorical_cols_df:
    for col in categorical_cols_df:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

df.dtypes


Age                         int64
Attrition                   int64
BusinessTravel              int64
DailyRate                   int64
Department                  int64
DistanceFromHome            int64
Education                   int64
EducationField              int64
EnvironmentSatisfaction     int64
Gender                      int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobRole                     int64
JobSatisfaction             int64
MaritalStatus               int64
MonthlyIncome               int64
MonthlyRate                 int64
NumCompaniesWorked          int64
OverTime                    int64
PercentSalaryHike           int64
PerformanceRating           int64
RelationshipSatisfaction    int64
StockOptionLevel            int64
TotalWorkingYears           int64
TrainingTimesLastYear       int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLast

In [24]:
import pandas as pd
import numpy as np
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Your existing DataFrame (load or assume named df)
#df = pd.read_csv('ibm_hr_with_features.csv')  # example filename

num_rows = df.shape[0]

# Define plausible categories for each sensitive column
genders = ['Male', 'Female', 'Other']
religions = ['Christianity', 'Islam', 'Hinduism', 'Buddhism', 'Judaism', 'None', 'Other']
races = ['White', 'Black or African American', 'Asian', 'Hispanic or Latino', 'Native American', 'Other']
marital_statuses = ['Single', 'Married', 'Divorced', 'Widowed', 'Separated']

# Generate random values for each column
df['gender'] = np.random.choice(genders, size=num_rows, p=[0.48, 0.48, 0.04])
df['religion'] = np.random.choice(religions, size=num_rows, p=[0.3, 0.25, 0.2, 0.1, 0.05, 0.05, 0.05])
df['race'] = np.random.choice(races, size=num_rows, p=[0.5, 0.13, 0.18, 0.12, 0.03, 0.04])
df['marital_status'] = np.random.choice(marital_statuses, size=num_rows, p=[0.4, 0.45, 0.1, 0.03, 0.02])

# Verify the result

print(df[['Age','gender', 'religion', 'race', 'marital_status']].head())
df.dtypes

# Save or continue processing as needed
#df.to_csv('ibm_hr_with_sensitive_features.csv', index=False)


   Age  gender      religion                race marital_status
0   41  Female         Islam               White         Single
1   49  Female  Christianity  Hispanic or Latino         Single
2   37   Other       Judaism               Asian        Married
3   33  Female         Islam  Hispanic or Latino       Divorced
4   27  Female      Buddhism               Other       Divorced


Age                          int64
Attrition                    int64
BusinessTravel               int64
DailyRate                    int64
Department                   int64
DistanceFromHome             int64
Education                    int64
EducationField               int64
EnvironmentSatisfaction      int64
Gender                       int64
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                      int64
JobSatisfaction              int64
MaritalStatus                int64
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
OverTime                     int64
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole  

In [25]:
# One-hot encode sensitive features for simplicity (alternative: label encoding or embedding)
df_sensitive = pd.get_dummies(df[sensitive_cols])

# For modeling, exclude sensitive features from X but keep them separately for fairness evaluation
X = df.drop(columns=[target] + sensitive_cols)  # all features except sensitive and target
y = df[target]

# Keep sensitive features as a DataFrame for Fairlearn
S = df[sensitive_cols]

# Train/test split (stratify on target)
X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(
    X, y, S, test_size=0.2, stratify=y, random_state=42
)


In [30]:
group_counts = pd.concat([y_train, S_train], axis=1).groupby(sensitive_cols)[target].value_counts().unstack(fill_value=0)
print(group_counts)


Attrition                                                 0  1
gender religion race                      marital_status      
Female Buddhism Asian                     Divorced        2  0
                                          Married         5  1
                                          Separated       1  0
                                          Single          3  2
                Black or African American Married         2  0
...                                                      .. ..
Other  None     Black or African American Single          1  0
       Other    Black or African American Divorced        1  0
                Hispanic or Latino        Married         1  0
                                          Single          0  1
                White                     Married         1  0

[259 rows x 2 columns]


In [32]:
#Filter out degenerate sensitive groups (at least 2 classes in group)
grouped = pd.concat([y_train, S_train], axis=1).groupby(sensitive_cols)
valid_groups = grouped.filter(lambda x: x[target].nunique() > 1)

X_train_filtered = X_train.loc[valid_groups.index]
y_train_filtered = y_train.loc[valid_groups.index]
S_train_filtered = S_train.loc[valid_groups.index]


Approach 1: Using ExponentiatedGradient 

In [41]:
base_clf = RandomForestClassifier(random_state=42)
base_clf.fit(X_train, y_train)

y_pred = base_clf.predict(X_test)
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))
print("Baseline ROC AUC:", roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

Baseline Accuracy: 0.8435374149659864
Baseline ROC AUC: 0.789042983891808


In [43]:
# Combine sensitive features into a single "group" series of tuples for Fairlearn
# Wrap the base estimator with Fairlearn’s constraint
mitigator = ExponentiatedGradient(
    estimator=RandomForestClassifier(random_state=42),
    constraints=DemographicParity()
)

mitigator.fit(X_train_filtered, y_train_filtered, sensitive_features=S_train_filtered)

y_pred_fair = mitigator.predict(X_test)

print("ExponentiatedGradient Mitigator Results:")
print("Fairness-Adjusted Accuracy:", accuracy_score(y_test, y_pred_fair))

# For ROC AUC, fallback to base classifier inside mitigator (approximate)

# Use base_clf for predict_proba as approximation (no mitigation)
y_proba_base = base_clf.predict_proba(X_test)[:, 1]

print("Approximate ROC AUC via unmitigated base classifier:", roc_auc_score(y_test, y_proba_base))


c:\Users\ctrid\AppData\Local\Programs\Python\Python311\Lib\site-packages\fairlearn\reductions\_exponentiated_gradient\_lagrangian.py:256: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.gammas[h_idx] = h_gamma
c:\Users\ctrid\AppData\Local\Programs\Python\Python311\Lib\site-packages\fairlearn\reductions\_exponentiated_gradient\_lagrangian.py:257: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.lambdas[h_idx] = lambda_vec.copy()
c:\Users\ctrid\AppData\Local\Programs\Python\Python311\Lib\site-packages\fairlearn\reductions\_ex

ExponentiatedGradient Mitigator Results:
Fairness-Adjusted Accuracy: 0.8469387755102041
Approximate ROC AUC via unmitigated base classifier: 0.789042983891808


c:\Users\ctrid\AppData\Local\Programs\Python\Python311\Lib\site-packages\fairlearn\reductions\_exponentiated_gradient\exponentiated_gradient.py:346: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pred[t] = np.zeros(len(X))
c:\Users\ctrid\AppData\Local\Programs\Python\Python311\Lib\site-packages\fairlearn\reductions\_exponentiated_gradient\exponentiated_gradient.py:346: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pred[t] = np.zeros(len(X))


Apprroach 2: Using ThresholdOptimizer for Postprocessing (Supports predict_proba)
ThresholdOptimizer is a postprocessing method that adjusts classification thresholds per group to satisfy fairness constraints.

In [45]:
base_clf = RandomForestClassifier(random_state=42)
base_clf.fit(X_train_filtered, y_train_filtered)

postprocessor = ThresholdOptimizer(
    estimator=base_clf,
    constraints="demographic_parity",
    predict_method='predict_proba'
)

postprocessor.fit(X_train_filtered, y_train_filtered, sensitive_features=S_train_filtered)

# Predict with fairness adjustments
y_pred_post = postprocessor.predict(X_test, sensitive_features=S_test)

print("Postprocessing Fairness Accuracy:", accuracy_score(y_test, y_pred_post))

# Use base classifier probabilities for approximate ROC AUC (no fairness adjustment)
y_proba_base = base_clf.predict_proba(X_test)[:, 1]

print("Base Classifier ROC AUC (no fairness adjustment):", roc_auc_score(y_test, y_proba_base))


Postprocessing Fairness Accuracy: 0.7891156462585034
Base Classifier ROC AUC (no fairness adjustment): 0.7803428374536998


In [ ]:
#Use the sensitive features DataFrame directly
group_test = S_test  # S_test is the sensitive features DataFrame used earlier

metric_frame = MetricFrame(
    metrics={
        'accuracy': accuracy_score,
        'selection_rate': selection_rate,
        'true_positive_rate': true_positive_rate,
        'false_positive_rate': false_positive_rate
    },
    y_true=y_test,
    y_pred=y_pred_fair,
    sensitive_features=group_test
)

print("Fairness metrics by group:")
print(metric_frame.by_group)


Fairness metrics by group:
                                      accuracy  selection_rate  \
gender religion race  marital_status                             
Female Buddhism Asian Divorced             NaN             NaN   
                      Married              1.0             0.0   
                      Separated            NaN             NaN   
                      Single               0.0             0.0   
                      Widowed              NaN             NaN   
...                                        ...             ...   
Other  Other    White Divorced             NaN             NaN   
                      Married              NaN             NaN   
                      Separated            NaN             NaN   
                      Single               NaN             NaN   
                      Widowed              NaN             NaN   

                                      true_positive_rate  false_positive_rate  
gender religion race  marital_stat

In [ ]:
#Combine multiple sensitive features into a single string group label

group_test = S_test.astype(str).agg('-'.join, axis=1)

metric_frame = MetricFrame(
    metrics={
        'accuracy': accuracy_score,
        'selection_rate': selection_rate,
        'true_positive_rate': true_positive_rate,
        'false_positive_rate': false_positive_rate
    },
    y_true=y_test,
    y_pred=y_pred_fair,
    sensitive_features=group_test
)

print("Fairness metrics by group:")
print(metric_frame.by_group)


Fairness metrics by group:
                                                  accuracy  selection_rate  \
sensitive_feature_0                                                          
Female-Buddhism-Asian-Married                     1.000000             0.0   
Female-Buddhism-Asian-Single                      0.000000             0.0   
Female-Buddhism-Black or African American-Single  1.000000             0.0   
Female-Buddhism-Hispanic or Latino-Single         1.000000             0.0   
Female-Buddhism-White-Married                     0.888889             0.0   
...                                                    ...             ...   
Other-Buddhism-White-Divorced                     1.000000             0.0   
Other-Christianity-White-Single                   0.500000             0.0   
Other-Hinduism-White-Divorced                     1.000000             0.0   
Other-Judaism-Asian-Married                       0.000000             0.0   
Other-Judaism-Hispanic or Latino-Sing

Advance Data Prep -

In [54]:
import pandas as pd

df['ReciprocalYearsAtCompany'] = 1 / (df['YearsAtCompany'] + 1)
df['RoleVsTenureRatio'] = df['YearsInCurrentRole'] / (df['YearsAtCompany'] + 1)

df['TenureBucket'] = pd.cut(df['YearsAtCompany'], bins=[0,1,3,5,10,20], labels=False)
df['PromotionRecencyBucket'] = pd.cut(df['YearsSinceLastPromotion'], bins=[-1,0,1,3,5,20], labels=False)

df['RoleStabilityRatio'] = df['YearsInCurrentRole'] / (df['YearsAtCompany'] + 1)
df['ManagerLoyaltyRatio'] = df['YearsWithCurrManager'] / (df['YearsAtCompany'] + 1)

df['PromotionVelocity'] = df['YearsAtCompany'] / (df['YearsSinceLastPromotion'] + 1)
df['RoleChangeVelocity'] = df['YearsAtCompany'] / (df['YearsInCurrentRole'] + 1)

df['IsRecentPromotion'] = (df['YearsSinceLastPromotion'] <= 1).astype(int)


Feature Selection using Lasso

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

features = df.drop(columns=['Attrition', 'YearsAtCompany', 'YearsSinceLastPromotion','YearsInCurrentRole','YearsWithCurrManager'])  # drop targets and dates
target = df['Attrition']

# Separate categorical and numeric columns
cat_cols = features.select_dtypes(include=['object', 'category']).columns
num_cols = features.select_dtypes(include=['number']).columns

# One-hot encode categorical columns
features_encoded = pd.get_dummies(features, columns=list(cat_cols), drop_first=True)

print(features_encoded.dtypes.value_counts())
print(features_encoded.columns[features_encoded.dtypes == 'object'])


int64      28
bool       17
float64     7
Name: count, dtype: int64
Index([], dtype='object')


In [65]:
print(features_encoded.isnull().sum())


Age                                 0
BusinessTravel                      0
DailyRate                           0
Department                          0
DistanceFromHome                    0
Education                           0
EducationField                      0
EnvironmentSatisfaction             0
Gender                              0
HourlyRate                          0
JobInvolvement                      0
JobLevel                            0
JobRole                             0
JobSatisfaction                     0
MaritalStatus                       0
MonthlyIncome                       0
MonthlyRate                         0
NumCompaniesWorked                  0
OverTime                            0
PercentSalaryHike                   0
PerformanceRating                   0
RelationshipSatisfaction            0
StockOptionLevel                    0
TotalWorkingYears                   0
TrainingTimesLastYear               0
WorkLifeBalance                     0
ReciprocalYe

In [66]:
features_imputed = features_encoded.copy()

# For numeric columns, fill NaNs with column mean
for col in features_imputed.columns:
    if features_imputed[col].dtype in ['float64', 'int64']:
        features_imputed[col].fillna(features_imputed[col].mean(), inplace=True)


C:\Users\ctrid\AppData\Local\Temp\ipykernel_17316\3371284272.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  features_imputed[col].fillna(features_imputed[col].mean(), inplace=True)


In [67]:
print(features_imputed.isnull().sum())

Age                               0
BusinessTravel                    0
DailyRate                         0
Department                        0
DistanceFromHome                  0
Education                         0
EducationField                    0
EnvironmentSatisfaction           0
Gender                            0
HourlyRate                        0
JobInvolvement                    0
JobLevel                          0
JobRole                           0
JobSatisfaction                   0
MaritalStatus                     0
MonthlyIncome                     0
MonthlyRate                       0
NumCompaniesWorked                0
OverTime                          0
PercentSalaryHike                 0
PerformanceRating                 0
RelationshipSatisfaction          0
StockOptionLevel                  0
TotalWorkingYears                 0
TrainingTimesLastYear             0
WorkLifeBalance                   0
ReciprocalYearsAtCompany          0
RoleVsTenureRatio           

In [68]:
X_to_scale = features_imputed  # This should be your fully encoded DataFrame
# Scale numeric columns only, or scale all (after encoding)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_to_scale)


In [ ]:


# L1 logistic regression for feature importance
lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=5000, random_state=42)
lasso.fit(X_scaled, target)

# Select features with non-zero coefficients
selected_features = features_imputed.columns[lasso.coef_[0] != 0]
print("Selected features via L1:", list(selected_features))


Selected features via L1: ['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'ReciprocalYearsAtCompany', 'RoleVsTenureRatio', 'TenureBucket', 'PromotionRecencyBucket', 'RoleStabilityRatio', 'ManagerLoyaltyRatio', 'PromotionVelocity', 'RoleChangeVelocity', 'gender_Other', 'religion_Christianity', 'religion_Hinduism', 'religion_Islam', 'religion_Judaism', 'religion_Other', 'race_Black or African American', 'race_Hispanic or Latino', 'race_Native American', 'race_Other', 'race_White', 'marital_status_Married', 'marital_status_Separated', 'marital_status_Single', 'marital_status_Widowed']


In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import shap

# Assume 'df' is your full DataFrame with target and features already prepared
target = df['Attrition']

# Use only the selected features from the L1 logistic regression step
selected_features = ['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome',
                     'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
                     'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
                     'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime',
                     'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
                     'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
                     'ReciprocalYearsAtCompany', 'RoleVsTenureRatio', 'TenureBucket', 'PromotionRecencyBucket',
                     'RoleStabilityRatio', 'ManagerLoyaltyRatio', 'PromotionVelocity', 'RoleChangeVelocity',
                     'gender_Other', 'religion_Christianity', 'religion_Hinduism', 'religion_Islam',
                     'religion_Judaism', 'religion_Other', 'race_Black or African American',
                     'race_Hispanic or Latino', 'race_Native American', 'race_Other', 'race_White',
                     'marital_status_Married', 'marital_status_Separated', 'marital_status_Single',
                     'marital_status_Widowed']

features_selected = df[selected_features]

# Split data into train and test sets with stratification on target
X_train, X_test, y_train, y_test = train_test_split(
    features_selected, target, test_size=0.2, stratify=target, random_state=42
)

# Scale features (important for many models and consistent feature scales)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest Classifier (robust and performs well on structured data)
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # probabilities for ROC AUC

# Evaluate model performance
print("Classification report on test set:")
print(classification_report(y_test, y_pred))

print(f"Test ROC AUC: {roc_auc_score(y_test, y_proba):.4f}")

# Optional: cross-validation scores for more robust performance estimation
cv_scores = cross_val_score(model, scaler.transform(features_selected), target, cv=5, scoring='roc_auc')
print(f"5-fold CV ROC AUC scores: {cv_scores}")
print(f"Mean CV ROC AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# -----------------------------
# SHAP Explainability

# Initialize SHAP TreeExplainer for Random Forest
explainer = shap.TreeExplainer(model)

# Calculate SHAP values on test set
shap_values = explainer.shap_values(X_test_scaled)

# Global feature importance plot (summary)
shap.summary_plot(shap_values[1], X_test, feature_names=selected_features)

# Optional: Force plot for single prediction explanation (interactive)
# shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], X_test.iloc[0,:], feature_names=selected_features)


KeyboardInterrupt: 